## Pengenalan Image Processing dengan Python
---
### 1. Import module yang diperlukan

In [ ]:
import numpy as np
import pandas as pd
import cv2 as cv 
from skimage import io
from PIL import Image 
import matplotlib.pylab as plt

---
### Tugas 1: Baca gambar dari URL dan tampilkan
> Cari gambar dari google, lalu gunakan url address dari gambar tersebut untuk melakukan operasi di bawah ini dengan menghapus tanda komentarnya.

In [ ]:
urls = ["url gambar.jpg"]

def cv_imshow(img):
    plt.figure(figsize = (15,15))
    plt.imshow(img)
    plt.xticks([]), plt.yticks([])
    plt.show()

for url in urls:
    myImg = io.imread(url)  
    cv_imshow(cv.cvtColor(myImg, cv.COLOR_BGR2RGB))

---
### Tugas 2: Tampilkan gambar anda dalam grayscale dan buat histogramnya

In [ ]:
image = io.imread(urls[0])

In [ ]:
plt.hist(image.ravel(),bins = 256, range = [0,256]) 
plt.show()

In [ ]:
color = ('b','g','r')
for i,col in enumerate(color):
    histr = cv.calcHist([image],[i],None,[256],[0,256])
    plt.plot(histr,color = col)
    plt.xlim([0,256])
plt.show()

In [ ]:
myGrayImg = cv.cvtColor(myImg, cv.COLOR_BGR2GRAY)
cv_imshow(myGrayImg)

---
### Tugas 3: Temukan kontur gambar Anda sendiri

In [ ]:
plt.contour(myGrayImg, origin = "image")

In [ ]:
ret, thresh = cv.threshold(myGrayImg,100,200,0)
contours, hierarchy = cv.findContours(thresh, cv.RETR_TREE, cv.CHAIN_APPROX_SIMPLE)
cv.drawContours(image, contours, -1, (0, 255, 0), 3)
plt.imshow(image)

---
### Tugas 4: Cobalah beberapa operasi matematika pada gambar Anda

In [ ]:
im3 = (100.0/255)*myGrayImg + 100
cv_imshow(im3)

In [ ]:
im4 = 255.0*(myGrayImg/255.0)**2
cv_imshow(im4)

---
### Tugas 5: Lakukan pemerataan histogram pada gambar grayscale Anda

In [ ]:
# Fungsi dari histogram equalization
def histeq(im, nbr_bins = 256):
  """ Persamaan histogram dari citra grayscale.  """
  # dapatkan histogram gambar
  imhist, bins = np.histogram(im.flatten(), nbr_bins, [0, 256])
  cdf = imhist.cumsum() # fungsi distribusi kumulatif
  cdf = imhist.max()*cdf/cdf.max()  #normalisasi
  cdf_mask = np.ma.masked_equal(cdf, 0)
  cdf_mask = (cdf_mask - cdf_mask.min())*255/(cdf_mask.max()-cdf_mask.min())
  cdf = np.ma.filled(cdf_mask,0).astype('uint8')
  return cdf[im.astype('uint8')]

# terapkan fungsi pada gambar gelap Anda untuk meningkatkan kontras
# kita dapat mengamati bahwa kontras latar belakang hitam telah meningkat
im5 = histeq(im4)
cv_imshow(im5)

In [ ]:
plt.hist(im4.ravel(),bins = 256, range = [0, 256])
plt.show()

In [ ]:
plt.hist(im5.ravel(),bins = 256, range = [0, 256])
plt.show()

---
### Tugas 6: Hasilkan transformasi fourier dari gambar grayscale Anda

In [ ]:
# Buramkan gambar grayscale dengan filter Guassian dengan ukuran kernel 10
imBlur = cv.blur(myGrayImg,(5,5))
# Ubah gambar menjadi domain frekuensi
f = np.fft.fft2(imBlur)
# Bawa komponen frekuensi-nol ke tengah
fshift = np.fft.fftshift(f)
magnitude_spectrum = 30*np.log(np.abs(fshift))

plt.subplot(121),plt.imshow(imBlur, cmap = 'gray')
plt.title('Input Image'), plt.xticks([]), plt.yticks([])
plt.subplot(122),plt.imshow(magnitude_spectrum, cmap = 'gray')
plt.title('Magnitude Spectrum'), plt.xticks([]), plt.yticks([])
plt.show()

---
### 6. Menemukan Tepi dengan Highpass Filtering di FFT
Bagian ini mendemonstrasikan melakukan high pass filter untuk menghilangkan komponen frekuensi rendah, sehingga menghasilkan gambar yang tajam yang berisi tepinya.

In [ ]:
rows, cols = imBlur.shape
crow,ccol = round(rows/2) , round(cols/2)
# hilangkan frekuensi rendah dengan ukuran persegi panjang 10
fshift[crow-10:crow+10, ccol-10:ccol+10] = 0
f_ishift = np.fft.ifftshift(fshift)
img_back = np.fft.ifft2(f_ishift)
img_back = np.abs(img_back)

plt.figure(figsize=([20, 20]))
plt.subplot(131),plt.imshow(imBlur, cmap = 'gray')
plt.title('Input Image'), plt.xticks([]), plt.yticks([])
plt.subplot(132),plt.imshow(img_back, cmap = 'gray')
plt.title('Gambar setelah HPF'), plt.xticks([]), plt.yticks([])
plt.subplot(133),plt.imshow(img_back)
plt.title('Hasil dalam JET'), plt.xticks([]), plt.yticks([])
plt.show()